In [ ]:
import pandas as pd
import numpy as np
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data=pd.read_csv('/content/abusive_sarcasm_data.csv')

In [ ]:
abusivedata=pd.read_csv('/content/abusive_data.csv')

In [ ]:
sarcasmdata=pd.read_csv('/content/sarcasm_data.csv')

In [ ]:
import sklearn.utils
abusivedata = sklearn.utils.shuffle(abusivedata)
abusivedata=abusivedata.reset_index()
abusivedata=abusivedata.drop('index',axis=1)
abusivedata

,text,toxic,sarcastic
0,brobdingrag gullivers travels is sitting in f...,0,0
1,I hope we see less of letter boxes on the stre...,1,0
2,ttonyb has no life whatsoever becides wikiped...,0,0
3,What bella thorne is donig is literal prostitu...,1,0
4,no what i m telling you is that you re the ...,0,0
...,...,...,...
204552,indeed you re probably on to something i thi...,0,0
204553,ive xome to ask to expand the article,0,0
204554,sigujana sigujana is not recognised in the zu...,0,0
204555,"@user so, what are you saying, blacks = crimin...",1,0


In [ ]:
import sklearn.utils
data = sklearn.utils.shuffle(data)
data=data.reset_index()
data=data.drop('index',axis=1)
data

,text,toxic,sarcastic
0,be more respectful of other editors tuvalkin ...,0,0
1,heh get out there and write writing is good ...,0,0
2,talk contribs,0,0
3,with these citation needed troll,0,0
4,You're a lot more respectful towards Republica...,0,1
...,...,...,...
409109,I made a mahogany pulpit Bully!,0,0
409110,I wonder how long reddit will allow this story...,0,1
409111,@thejournalist44 At least 3. Late tho. Cuz adr...,1,0
409112,welcome hello and welcome to wikipedia thank...,0,0


In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text) 
    text = re.sub('\ï½ ', '', text)
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
        
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)
    
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))


In [ ]:
data['processed_text'] = data['text'].apply(lambda x: finalpreprocess(x))

In [ ]:
abusivedata['processed_text'] = abusivedata['text'].apply(lambda x: finalpreprocess(x))

In [ ]:
sarcasmdata['processed_text'] = sarcasmdata['text'].apply(lambda x: finalpreprocess(x))

In [ ]:
data

,text,toxic,sarcastic,processed_text
0,be more respectful of other editors tuvalkin ...,0,0,respectful editor tuvalkin absolutely unaccept...
1,heh get out there and write writing is good ...,0,0,heh get write write good certainly well admin ...
2,talk contribs,0,0,talk contribs
3,with these citation needed troll,0,0,citation need troll
4,You're a lot more respectful towards Republica...,0,1,lot respectful towards republicans applaud lev...
...,...,...,...,...
409109,I made a mahogany pulpit Bully!,0,0,make mahogany pulpit bully
409110,I wonder how long reddit will allow this story...,0,1,wonder long reddit allow story remain racism d...
409111,@thejournalist44 At least 3. Late tho. Cuz adr...,1,0,thejournalist least late tho cuz adrenaline em...
409112,welcome hello and welcome to wikipedia thank...,0,0,welcome hello welcome wikipedia thank contribu...


In [ ]:
data.to_csv('processed_abusive_sarcasm_data.csv', index=False)

In [ ]:
abusivedata

,text,toxic,sarcastic,processed_text
0,brobdingrag gullivers travels is sitting in f...,0,0,brobdingrag gulliver travel sit front letter c...
1,I hope we see less of letter boxes on the stre...,1,0,hope see less letter box street woman drive in...
2,ttonyb has no life whatsoever becides wikiped...,0,0,ttonyb life whatsoever becides wikipedia pht n...
3,What bella thorne is donig is literal prostitu...,1,0,bella thorne donig literal prostitution screen...
4,no what i m telling you is that you re the ...,0,0,tell one posting false statement wikipedia sto...
...,...,...,...,...
204552,indeed you re probably on to something i thi...,0,0,indeed probably something think fourth generat...
204553,ive xome to ask to expand the article,0,0,ive xome ask expand article
204554,sigujana sigujana is not recognised in the zu...,0,0,sigujana sigujana recognise zulu king list nev...
204555,"@user so, what are you saying, blacks = crimin...",1,0,user say black criminal blm teamd


In [ ]:
abusivedata.to_csv('processed_abusive_data.csv', index=False)

In [ ]:
sarcasmdata

,text,sarcastic,toxic,processed_text
0,What about those of us who aren't going to tou...,1,0,u go touch fo least month banishment yep shut ...
1,"It is about abortion. Oh right, which has noth...",1,0,abortion oh right nothing woman health
2,can guess what it is? Should be pretty easy to...,1,0,guess pretty easy figure dr write patient diag...
3,You could get Enhanced Targeting and attack in...,1,0,could get enhanced target attack infantry miss...
4,Can someone tweet this to official PMO account...,1,0,someone tweet official pmo account ask explana...
...,...,...,...,...
204552,"PC gamers, what are some awesome games that do...",1,0,pc gamers awesome game require powerhouse pc r...
204553,Scott Walker Explains Why He Thinks Obama Dese...,0,0,scott walker explain think obama deserves blam...
204554,What happens if other animals discover GOD? In...,0,0,happens animal discover god edgy atheist
204555,"I know! Cincy has one, that would be the close...",0,0,know cincy one would closest tell people get t...


In [ ]:
sarcasmdata.to_csv('processed_sarcasm_data.csv', index=False)